In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from datetime import datetime


df = pd.read_csv("../data/train.csv")

In [11]:
df['Dates'] = pd.to_datetime(df['Dates'])
df['Hour'] = df['Dates'].dt.hour
df['Day'] = df['Dates'].dt.day
df['Month'] = df['Dates'].dt.month
df['Weekday'] = df['Dates'].dt.weekday
#الاجازه يوم الاتنين 

In [12]:
categorical_cols = ['DayOfWeek', 'PdDistrict', 'Resolution']
df_encoded = pd.get_dummies(df[categorical_cols], drop_first=True)
#تحويل الاعمده النصيه لارقام 

In [ ]:
features = pd.concat([
    df[['X', 'Y', 'Hour', 'Day', 'Month', 'Weekday']],
    df_encoded
], axis=1)

#بنعمل دمج لاعمدة

In [ ]:
scaler = MinMaxScaler()
features_scaled = pd.DataFrame(scaler.fit_transform(features), columns=features.columns)
#عملنا scaling للبيانات 
#علشان تقبي في interval 0-1

In [15]:
features_scaled.to_csv("../data/preprocessed_data.csv", index=False)
print(" Data preprocessing complete. File saved as 'data/preprocessed_data.csv'")

 Data preprocessing complete. File saved as 'data/preprocessed_data.csv'
